## Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  

## Equipe:

1. Afonso Henrique Torres Lucas | ifonso.developer@gmail.com | iFonso - 2215080047
2. Emanuelle Rocha Marreira | emanuellemarreira@gmail.com | emanuellemarreira - 2215080035
3. Erik Gustavo Lima de Oliveira | erik.exatas10@gmail.com | ErikExatas - 2115080049
4. David Augusto De Oliveira E Silva | david-augusto-silva - 2115080006
5. Ítalo Ferreira Fonseca
6. João Vitor Silva De Carvalho
7. Lilian Iazzai De Souza Oliveira | lilianiazzai@gmail.com | lilianiazzai - 2215080018
8. Vitor Nascimento Aguiar | Vtaguiar1909 - 2115080055

- **Github**: https://github.com/emanuellemarreira/image-authenticity-classification/tree/parte1

In [ ]:
# bibliotecas
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Descrição do problema

A qualidade de imagens geradas por inteligência artificial tem rapidamente aumentado, levando a preocupação da autenticidade e da segurança dos dados levantados. Isso ocorre pelo aumento gradativo de melhores modelos de inteligência, gerando fotos fiéis ao que o olho humano intepreta como real. Essas imagens geradas artifialmente estão no pódio a disputar com imagens geradas por humanos. Com isso, o dataset CIFAKE foi proposto como uma base para a tarefa de classificação binária de imagens: distinguir entre imagens reais e imagens sintéticas geradas por IA. Ele combina 60.000 imagens reais do dataset CIFAR-10 (um conjunto de imagens de baixa resolução amplamente utilizado em tarefas de aprendizado de máquina) com 60.000 imagens sintéticas geradas por modelos de difusão latente (Stable Diffusion).

As imagens têm resolução de 32×32 pixels e estão distribuídas em 10 classes: avião, automóvel, pássaro, gato, cervo, cachorro, sapo, cavalo, navio e caminhão.

O objetivo do modelo é classificar cada imagem como real ou falsa (gerada por IA).

## Pré-processamento

In [2]:
transform = transforms.Compose([
    # Resize da imagem pra 32x32 (pixels)
    transforms.Resize((32, 32)),
    # PIL -> Tensor[canal, altura, largura] + Normalização de pixels
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root="../data/train", transform=transform)
test_data = datasets.ImageFolder(root="../data/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

Inicialmente, foi definida uma transformação para os dados carregados, redimensionando todas as imagens do dataset para 32×32 pixels. Em seguida, é aplicada outra transformação que converte as imagens do formato padrão PIL `[altura, largura, canais]` para o formato `[canais, altura, largura]` utilizado pelo PyTorch, além de normalizar os valores de pixel de 0–255 para o intervalo $[0.0, 1.0]$. No fim temos dataloaders de teste e de treino com batche de 32 para facilitar a iteração sobre o conjunto de dados.

Nosso conjunto de dados é composto por duas classes, de imagens reais e de imagens geradas artificialente, a seguir temos as classes:

In [3]:
print(train_data.classes)
print(train_data.class_to_idx)

['FAKE', 'REAL']
{'FAKE': 0, 'REAL': 1}


In [4]:
print(f"Dados:\n\ntreino: {len(train_data)} \nteste: {len(test_data)}")

Dados:

treino: 100000 
teste: 20000


In [5]:
fake_class_data_count = sum(1 for _, label in train_data if label == 1)
real_class_data_count = sum(1 for _, label in train_data if label == 0)

print(f"Quantidade de dados de treino:\nlabel 'REAL':{real_class_data_count} \nlabel 'FAKE':{fake_class_data_count}")

Quantidade de dados de treino:
label 'REAL':50000 
label 'FAKE':50000
